In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Base URL for historical snapshots
base_url = "https://coinmarketcap.com/historical/"

# List of dates to scrape, you could make this dynamic based on available dates in base_url
dates = ["20130428", "20130505", "20130512"]  # Add more dates in the format 'YYYYMMDD'

# Prepare an empty list to store data
all_data = []

# Loop through each date
for date in dates:
    url = f"{base_url}{date}/"
    print(f"Scraping {url}")
    
    # Send request to get page content
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to retrieve data for {date}")
        continue

    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find the table or div containing asset data (check HTML structure of each snapshot page)
    rows = soup.select("table tbody tr")  # Modify selector as per actual structure
    
    # Loop through the rows and extract the top 125 assets
    for i, row in enumerate(rows[:125]):  # Limit to top 125 rows
        try:
            symbol = row.select_one(".cmc-table__cell--sort-by__symbol").text.strip()
            market_cap = row.select_one(".cmc-table__cell--sort-by__market-cap").text.strip()
            
            # Append to the all_data list
            all_data.append({
                "Date": date,
                "Rank": i + 1,
                "Symbol": symbol,
                "MarketCap": market_cap
            })
        except AttributeError:
            print(f"Missing data for rank {i + 1} on date {date}")
    
    # Delay to avoid hitting the server too fast
    time.sleep(1)  # Adjust as needed

# Convert to DataFrame and save to CSV
df = pd.DataFrame(all_data)
df.to_csv("coinmarketcap_historical_data.csv", index=False)
print("Data saved to coinmarketcap_historical_data.csv")

Scraping https://coinmarketcap.com/historical/20130428/
Scraping https://coinmarketcap.com/historical/20130505/
Scraping https://coinmarketcap.com/historical/20130512/
Data saved to coinmarketcap_historical_data.csv
